In [8]:
from gensim.models import Word2Vec, KeyedVectors

#if there is extra input at the top
removeExtras = True

class CreateWordEmbeddings:
    def __init__(self, path, sentences = []):
        self.embeddingModel = Word2Vec(min_count=1, epochs=100)
        self.path = path
        self.sentences = sentences
        self.size = None

    def process(self):
        self.embeddingModel.build_vocab(self.sentences)
        self.embeddingModel.train(self.sentences, total_examples=self.embeddingModel.corpus_count, epochs=self.embeddingModel.epochs)
        self.embeddingModel.wv.save_word2vec_format(self.path, binary=False)

        if removeExtras:
            data = None
            with open(self.path, 'r', encoding='utf-8') as fin:
                data = fin.read().splitlines(True)
                self.size = tuple(data[0].strip().split(' '))

            with open(self.path, 'w', encoding='utf-8') as fout:
                fout.truncate(0)
                data = data[1:]
                for i, line in enumerate(data):
                    # condition to remove blanks
                    if i == 1601: #to debug
                        pass
                    if not (line.startswith(' ') or line.startswith('\n') or line.startswith('\t')):
                        fout.writelines(line)

    def processSentences(self, sentences):
        self.sentences = sentences
        self.process()



    def getDictionarySize(self):
        return self.size


In [10]:
import json
import pandas as pd
import os

class limscParser:
    def __init__(self, dir, ref_path, jsonStr, dataType):
        self.dir = dir
        self.ref = open(ref_path, 'r', encoding='utf-8').read()
        self.dataType = dataType
        self.data = []
        self.tags = []
        self.vocab = []
        self.outdir = os.path.join(self.dir, "out") 
        self.embed = CreateWordEmbeddings("emb.txt")
        self.json_str = jsonStr


    def process(self):
        for filename in os.listdir(self.dir):
            # Check if the file has the desired extension
            if filename.endswith(".parquet"):
                # file parsing
                data = json.loads(pd.read_parquet(os.path.join(self.dir, filename)).to_json())
                
                if self.dataType == "NER_LIMSC":
                    # defined as per JSON string
                    # Note : this needs to be adapted according to the string parser
                    # todo
                    # forr limsc uncomment below
                    # refNerTags = json.loads(self.ref)[self.json_str]['features']['ner_tags']['feature']['names']
                    # for first trail
                    refNerTags = json.loads(self.ref)

                    # get sent tokens
                    self.data = [val for key,val in data["tokens"].items()]

                    # get ner tags
                    self.tags = [[refNerTags[str(tag)]  for tag in val] for key,val in data["ner_tags"].items()]

                    # for key,val in data["ner_tags"].items():
                    #     tags = []
                    #     for tag in val:
                    #         tags.append(refNerTags.get(str(tag)))
                    #     self.tags.append(tags)


                if not os.path.exists(self.outdir):
                    os.makedirs(self.outdir)

                if "test" in filename:
                    self.index = round(0.6 * len(self.data))
                    allData = self.data
                    allTags = self.tags

                    self.data = allData[:self.index]
                    self.tags = allTags[:self.index]
                    self.write(os.path.join(self.outdir, "testA.Conll"), "test")

                    self.data = allData[self.index: ]
                    self.tags = allTags[self.index: ]
                    self.write(os.path.join(self.outdir, "testB.Conll"), "test")

                    self.data = allData
                    self.tags = allTags

                elif "train" in filename:
                    self.write(os.path.join(self.outdir, "train.Conll"))

                elif "val" in filename:
                    self.write(os.path.join(self.outdir, "val.Conll"))
                
                else:
                    pass
        self.embed.processSentences(self.vocab)
        self.embed.getDictionarySize()
                        
        if self.dataType == "CONTEXT_LIMSC":
            pass

        pass

    def write(self, outPath, tag = "normal"):
        with open(outPath, 'w', encoding='utf-8') as f:
            f.write('-DOCSTART- -X- -X- O\n\n')
            # as we are only processing NER tags we can keep the mid blanks empty
            assert len(self.data) == len(self.tags)
            for i,val in enumerate(self.data):
                assert len(self.data[i]) == len(self.tags[i])
                if tag == "normal":
                    [f.write('{} {} {} {}\n'.format(key, "X", "X", self.tags[i][j])) for j, key in enumerate(val)]
                else:
                    [f.write('{}\n'.format(key)) for j, key in enumerate(val)]
                f.write('\n')
                self.vocab.append(val)



# lp = limscParser('conceptRecognition', 'conceptRecognition/dataset_infos.json', "limsc--concept-recognition", "NER_LIMSC")
# lp.process()


lp = limscParser('trailClassification/trail', 'trailClassification/tags.json', "limsc--concept-recognition", "NER_LIMSC")
lp.process()